In [1]:
# necessary packages
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random as rng

from skimage.measure import label
from skimage.exposure import histogram
from skimage import exposure
from skimage.filters import threshold_otsu
from skimage.segmentation import watershed
from skimage.morphology import white_tophat,black_tophat
from skimage.morphology import reconstruction,remove_small_holes,remove_small_objects

# optional package
# from scipy import ndimage as ndi
# from skimage.filters import sobel
# from skimage.filters import meijering

In [2]:
# preprocessing functions
def contrastStretch(img, a=0, b=255):
    c = np.min(img)
    d = np.max(img)
    if not (d-c):
        return img
    stretch = (b-a)/(d-c)    #stretch ratio
    return np.round((img-c)*stretch+a).astype(np.uint8)

def intensityHistogram(img):
    y,x = histogram(img)
    plt.figure(figsize=(8,6))         # plot the histogram
#     plot = plt.bar(x, y, width = 0.8, align='center')
    plot = plt.plot(x, y)
    plt.title('Intensity Histogram',fontsize=20)
    plt.ylabel('h(i)',fontsize=18)
    plt.xlabel('pixel intensity values',fontsize=18)
    return plot

def drawBoundingBox(image,img_labels,color=None):
    out_img = image.copy()
    out_img = contrastStretch(cv2.cvtColor(out_img,cv2.COLOR_GRAY2BGR)).astype(np.uint8)
    for label in np.unique(img_labels)[1:]:
        if color is None:
            label_color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
        else:
            label_color = color
        x, y, w, h = cv2.boundingRect((img_labels==label).astype(np.uint8))
        cv2.rectangle(out_img,(x,y),(x+w,y+h),label_color,2)
    return out_img

def fakeUltimate(img,iteration=True):
    S = np.ones((3,3))
    n_labels = np.unique(label(img)).shape[0]
    if n_labels<=1: # 1 label for background
        return img
    
    while True and iteration:
        new_img = cv2.erode(img, S)
        new_n_labels = np.unique(label(new_img)).shape[0]
        if new_n_labels<n_labels:
            break
        n_labels = new_n_labels
        img = new_img
        if type(iteration) is int:
            iteration-=1
    return img

def ultimateErosion(mask,iteration=True):
    labels = label(mask)
    retVal = np.zeros_like(labels)
    for i in np.unique(labels)[1:]:
        retVal+=fakeUltimate((labels==i).astype(np.uint8),iteration)
    return retVal

def removeSmallLabel(labels, t):
    for i in np.unique(labels):
        if np.sum(labels==i) < t:
            labels = np.where(labels==i,0,labels)
    return labels

def Segmentation(img,cell_size,binary_thresh=None,noise_size=5,label_thresh=0,blackHat=True,whiteHat=True):
    if blackHat:
        img = img+black_tophat(img,selem=np.ones((noise_size,noise_size)))
    if whiteHat:
        img = white_tophat(img,selem=np.ones((cell_size,cell_size)))
    img = cv2.GaussianBlur(img, (9,9), 1)
    img = contrastStretch(img)
    if binary_thresh is None:
        binary_thresh=threshold_otsu(img)
    mask = remove_small_objects(img>binary_thresh,label_thresh)
    Seed = ultimateErosion(mask)
    ws_labels = watershed(mask,markers=label(Seed),mask=mask)
    ws_labels = removeSmallLabel(ws_labels,label_thresh)
    return ws_labels

In [4]:
FluoPath = 'COMP9517 20T2 Group Project Image Sequences/Fluo-N2DL-HeLa/'
PhcPath = 'COMP9517 20T2 Group Project Image Sequences/PhC-C2DL-PSC/'
seqences = ['Sequence 1/','Sequence 2/','Sequence 3/','Sequence 4/']

In [7]:
# process all img and save labels in .npy file
# Fluo
for seq in seqences:
    print(f'------{seq}------')
    for imgName in sorted(os.listdir(FluoPath+seq)):
        print(imgName+'\r')
        img = cv2.cvtColor(cv2.imread(FluoPath+seq+imgName),cv2.COLOR_RGB2GRAY)
        img_label = Segmentation(img,cell_size=21,binary_thresh=0,label_thresh=100,blackHat=True,whiteHat=True)
        np.save('Fluo Label/'+seq+imgName,img_label)

------Sequence 1/------
t000.tif
t001.tif
t002.tif
t003.tif
t004.tif
t005.tif
t006.tif
t007.tif
t008.tif
t009.tif
t010.tif
t011.tif
t012.tif
t013.tif
t014.tif
t015.tif
t016.tif
t017.tif
t018.tif
t019.tif
t020.tif
t021.tif
t022.tif
t023.tif
t024.tif
t025.tif
t026.tif
t027.tif
t028.tif
t029.tif
t030.tif
t031.tif
t032.tif
t033.tif
t034.tif
t035.tif
t036.tif
t037.tif
t038.tif
t039.tif
t040.tif
t041.tif
t042.tif
t043.tif
t044.tif
t045.tif
t046.tif
t047.tif
t048.tif
t049.tif
t050.tif
t051.tif
t052.tif
t053.tif
t054.tif
t055.tif
t056.tif
t057.tif
t058.tif
t059.tif
t060.tif
t061.tif
t062.tif
t063.tif
t064.tif
t065.tif
t066.tif
t067.tif
t068.tif
t069.tif
t070.tif
t071.tif
t072.tif
t073.tif
t074.tif
t075.tif
t076.tif
t077.tif
t078.tif
t079.tif
t080.tif
t081.tif
t082.tif
t083.tif
t084.tif
t085.tif
t086.tif
t087.tif
t088.tif
t089.tif
t090.tif
t091.tif
------Sequence 2/------
t000.tif
t001.tif
t002.tif
t003.tif
t004.tif
t005.tif
t006.tif
t007.tif
t008.tif
t009.tif
t010.tif
t011.tif
t012.tif
t013.ti

In [ ]:
# process all img and save labels in .npy file
# PhC
for seq in seqences:
    print(f'------{seq}------')
    for imgName in sorted(os.listdir(PhcPath+seq)):
        print(imgName+'\r')
        img = cv2.cvtColor(cv2.imread(PhcPath+seq+imgName),cv2.COLOR_RGB2GRAY)
        img_label = Segmentation(img,cell_size=21,label_thresh=20,blackHat=False,whiteHat=True)
        cv2.imwrite('Phc Label/'+seq+imgName,img_label)